# Class 4: Hyperparameter Tuning and Mini-Project

**Week 6: Supervised Learning Algorithms**

## Overview
Welcome to Class 4! Today, we’ll wrap up Week 6 by learning about **hyperparameter tuning** using grid search and applying everything we’ve learned in a **mini-project**. You’ll build classifiers (logistic regression, KNN, decision trees) to predict Iris species, tune one model, and compare performance using evaluation metrics from Class 3. This is your chance to put it all together!

## Objectives
- Understand what hyperparameters are and why tuning matters.
- Learn how to use grid search for hyperparameter optimization.
- Complete a mini-project to build, evaluate, and compare classifiers.
- Gain hands-on experience with an end-to-end machine learning workflow.

## Agenda
1. Introduction to hyperparameters
2. Grid search for tuning
3. Mini-project: Build and compare classifiers

Let’s dive in!

## 1. Introduction to Hyperparameters

**Hyperparameters** are settings you choose before training a model, unlike parameters (e.g., weights) the model learns. They control how the model behaves.

**Examples**:
- **KNN**: `n_neighbors` (number of neighbors, *k*).
- **Decision Trees**: `max_depth` (how deep the tree can grow).
- **Logistic Regression**: `C` (regularization strength).

**Why tune?** The right hyperparameters improve performance. Bad choices can lead to overfitting (too complex) or underfitting (too simple).

**Question**: What might happen if `max_depth` is too large in a decision tree? (Hint: Think about Class 2.)

## 2. Grid Search for Tuning

**Grid search** is a method to systematically test combinations of hyperparameters to find the best ones.

**How it works**:
- Define a "grid" of possible values (e.g., `n_neighbors = [3, 5, 7]`).
- Train and evaluate the model for each combination using cross-validation.
- Pick the combination with the best performance (e.g., highest accuracy).

**Example**: For KNN, test `n_neighbors = [1, 3, 5]` and `weights = ['uniform', 'distance']`.

We’ll use scikit-learn’s `GridSearchCV` to automate this. Let’s try it later in the mini-project!

## 3. Mini-Project: Build and Compare Classifiers

For the mini-project, you’ll build classifiers to predict **Iris species** (setosa, versicolor, virginica) using the full Iris dataset (3 classes, all 4 features). You’ll train logistic regression, KNN, and decision trees, evaluate them with metrics from Class 3, and tune one model with grid search.

**Steps**:
1. Load and prepare the Iris dataset.
2. Train logistic regression, KNN, and decision tree models.
3. Evaluate models using accuracy, precision, recall, and F1-score.
4. Tune one model (e.g., KNN) with grid search.
5. Compare results and pick the best model.

Let’s start coding! Feel free to work in pairs or individually.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

# Load Iris dataset (full: 3 classes)
iris = load_iris()
X = iris.data
y = iris.target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the data
print("Feature names:", iris.feature_names)
print("Target names:", iris.target_names)
print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

**What did we do?**
- Loaded the full Iris dataset (150 samples, 4 features: sepal length, sepal width, petal length, petal width).
- Kept all 3 classes (setosa, versicolor, virginica).
- Split into 80% training (120 samples) and 20% testing (30 samples).

Let’s train the three models with default settings.

In [ ]:
# Train models
# Logistic Regression
lr = LogisticRegression(random_state=42, max_iter=200)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# Decision Tree
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print("Models trained! Ready to evaluate.")

Now, let’s evaluate all models using accuracy, precision, recall, and F1-score. Since this is multi-class, we’ll average metrics across classes.

In [ ]:
# Function to compute and print metrics (multi-class)
def print_metrics(y_true, y_pred, model_name):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted')
    rec = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f"{model_name}:")
    print(f"  Accuracy: {acc:.3f}")
    print(f"  Precision: {prec:.3f}")
    print(f"  Recall: {rec:.3f}")
    print(f"  F1-Score: {f1:.3f}")
    print()

# Compute metrics for all models
print_metrics(y_test, y_pred_lr, "Logistic Regression")
print_metrics(y_test, y_pred_knn, "KNN (k=3)")
print_metrics(y_test, y_pred_dt, "Decision Tree")

**Your turn!**
- Which model performs best based on accuracy? F1-score?
- Are precision and recall balanced? Why might they differ slightly?

Let’s visualize confusion matrices to see where errors occur.

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=iris.target_names, yticklabels=iris.target_names)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# Plot confusion matrices
plot_confusion_matrix(y_test, y_pred_lr, "Logistic Regression Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_knn, "KNN (k=3) Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_dt, "Decision Tree Confusion Matrix")

**Discussion**:
- Look at the confusion matrices. Which classes are confused most often (e.g., versicolor vs. virginica)?
- Which model has the fewest errors?

Now, let’s tune the KNN model using grid search to see if we can improve its performance.

In [ ]:
# Define parameter grid for KNN
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

# Initialize KNN
knn = KNeighborsClassifier()

# Set up grid search
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

# Run grid search
grid_search.fit(X_train, y_train)

# Print results
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_:.3f)

# Evaluate tuned model on test set
y_pred_knn_tuned = grid_search.predict(X_test)
print_metrics(y_test, y_pred_knn_tuned, "Tuned KNN")
plot_confusion_matrix(y_test, y_pred_knn_tuned, "Tuned KNN Confusion Matrix")

**Your turn!**
- Did the tuned KNN perform better than the default KNN (k=3)? Check accuracy and F1-score.
- Look at the confusion matrix. Are there fewer errors?

**Challenge**: Try tuning the decision tree instead! Copy the grid search code and use:
```python
param_grid = {'max_depth': [2, 3, 4, 5], 'min_samples_split': [2, 5, 10]}
dt = DecisionTreeClassifier()
```
Run it and compare results (optional).

## Wrap-Up

Today, you:
- Learned about **hyperparameters** and **grid search**.
- Completed a **mini-project** to predict Iris species.
- Trained, evaluated, and tuned classifiers (logistic regression, KNN, decision trees).
- Compared models using metrics and confusion matrices.

**Deliverable**:
- Share your findings: Which model performed best? Why? (Discuss in class or submit a short summary.)

**Homework**:
- Experiment with a different random seed (e.g., `random_state=123`) and see how results change.
- Explore the [scikit-learn GridSearchCV documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) (5-10 min).

**What’s Next?**
- Try other algorithms (e.g., SVM, random forests) or datasets (e.g., from Kaggle).
- Dive deeper into feature engineering or real-world applications.

Congratulations on completing Week 6! Any questions?